# Test Factor Models VaR and ES
This notebook demonstrates how to use:
- A single-factor model Value-at-Risk (VaR) and Expected Shortfall (ES)
- A Fama-French 3-factor model for VaR and ES

It uses historical return data and factor models to estimate portfolio risk.

In [22]:
import data_download as dd
import factor_models as fm
import pandas as pd
import numpy as np

##  Download data and form a portfolio

In [23]:
# Define tickers and download price data
tickers = ["MSFT", "NVDA", "AAPL"]
prices = dd.get_raw_prices(tickers, start="2024-01-01")


# Define portfolio — 3 shares of each asset
shares = pd.Series({"MSFT": 3, "NVDA": 3, "AAPL": 3})

# Compute daily value of each position
position_values = prices * shares

# Compute total portfolio value over time
portfolio_value_series = position_values.sum(axis=1)

# Compute asset weights (based on latest available prices)
latest_prices = prices.iloc[-1]
port_val = (shares * latest_prices).sum()
weights = (shares * latest_prices) / port_val

# Compute daily asset returns and portfolio returns
returns = prices.pct_change().dropna()
portfolio_returns = returns @ weights

# Combine historical values into one DataFrame
portfolio_df = position_values.copy()
portfolio_df["PORTFOLIO_TOTAL"] = portfolio_value_series

# Display results
portfolio_df.head()


,AAPL,MSFT,NVDA,PORTFOLIO_TOTAL
Date,,,,
2024-01-02,553.596222,1102.141754,144.449753,1800.187729
2024-01-03,549.451172,1101.339569,142.653419,1793.444160
2024-01-04,542.473022,1093.434540,143.939953,1779.847515
2024-01-05,540.296127,1092.869934,147.235691,1780.401752
2024-01-08,553.357681,1113.494019,156.700138,1823.551838


## Single-Factor VaR and ES

The Single-Factor model assumes that all asset returns are driven by a single common factor — typically the market index.  
This is also referred to as the **Sharpe Single-Index Model**, and is a special case of the delta-normal method.

Each asset’s return is modeled as:

$$
R_i = \alpha_i + \beta_i R_m + \varepsilon_i
$$

where:
- $R_m$ is the market return
- $\beta_i$ is the sensitivity of asset $i$ to the market
- $\varepsilon_i$ is the idiosyncratic (asset-specific) risk

The total variance of asset $i$ is:

$$
\sigma_i^2 = \beta_i^2 \sigma_m^2 + \sigma_{\varepsilon_i}^2
$$

The portfolio’s covariance matrix is approximated as:

$$
\Sigma = \beta \beta^T \cdot \sigma_m^2 + D
$$

where:
- $\beta$: vector of asset betas
- $\sigma_m^2$: variance of the market return
- $D$: diagonal matrix of idiosyncratic variances

Finally, the portfolio VaR at confidence level $\alpha$ is given by:

$$
\text{VaR}_{\alpha} = z_\alpha \cdot \sigma_p \cdot \text{PortfolioValue}
$$

and the Expected Shortfall (ES) is:

$$
\text{ES}_{\alpha} = \frac{\phi(z_\alpha)}{1 - \alpha} \cdot \sigma_p \cdot \text{PortfolioValue}
$$

where $z_\alpha$ is the quantile of the standard normal distribution and $\phi$ is the standard normal PDF.


In [24]:
# download SPY prices
spy = dd.get_raw_prices(["SPY"], start="2024-01-01")

# compute benchmark returns
benchmark = spy["SPY"].pct_change().dropna()

# align benchmark with asset returns
benchmark = benchmark.reindex(returns.index).ffill()

In [25]:
var_sf, es_sf, Sigma, betas, idio_var = fm.single_factor_var_es(
    returns=returns,
    benchmark=benchmark,
    weights=weights,
    port_val=port_val,
    confidence_level=0.99
)

# Print results
print("Single Factor Portfolio VaR (99%):", f"{var_sf:,.2f} USD")
print("Single Factor Portfolio ES  (99%):", f"{es_sf:,.2f} USD\n")

print("Covariance matrix:")
display(Sigma)

print("\n Asset betas relative to SPY:")
display(betas)

print("\n Idiosyncratic variances:")
display(idio_var)

Single Factor Portfolio VaR (99%): 85.34 USD
Single Factor Portfolio ES  (99%): 97.77 USD

Covariance matrix:


Ticker,AAPL,MSFT,NVDA
Ticker,,,
AAPL,0.000360,0.000156,0.000338
MSFT,0.000156,0.000234,0.000288
NVDA,0.000338,0.000288,0.001323



 Asset betas relative to SPY:


Ticker
AAPL    1.180196
MSFT    1.005505
NVDA    2.179345
dtype: float64


 Idiosyncratic variances:


Ticker
AAPL    0.000177
MSFT    0.000101
NVDA    0.000698
dtype: float64

## Fama-French 3-Factor VaR and ES

The Fama–French 3-factor model extends the CAPM by modeling asset returns using three sources of systematic risk:

- **Mkt_RF**: excess return of the market over the risk-free rate  
- **SMB** ("Small Minus Big"): return of small-cap stocks minus large-cap  
- **HML** ("High Minus Low"): return of high book-to-market stocks minus low  

Each asset’s excess return is regressed on these three factors to estimate its **exposures (betas)**. The model assumes:

$$
R_i - R_f = \alpha_i + \beta_{i1} \cdot \text{Mkt}_{RF} + \beta_{i2} \cdot \text{SMB} + \beta_{i3} \cdot \text{HML} + \varepsilon_i
$$

The estimated covariance matrix of asset returns is reconstructed as:

$$
\Sigma = B \cdot \Sigma_f \cdot B^T + D
$$

Where:
- $B$: matrix of asset betas  
- $\Sigma_f$: covariance matrix of factor returns  
- $D$: diagonal matrix of idiosyncratic variances  

The next code block performs the following steps:

1. Downloads the FF3 daily factor data from Ken French's database.
2. Computes **excess returns** for each asset over the risk-free rate.
3. Regresses each asset’s excess return on the three factors to obtain:
   - **Beta coefficients** (exposures to Mkt_RF, SMB, HML)
   - **Idiosyncratic variance** (residual risk)
4. Builds the **full covariance matrix** of asset returns using the factor structure.
5. Computes **portfolio volatility** using the covariance matrix and current weights.
6. Calculates:
   - **Value-at-Risk (VaR)** at the given confidence level
   - **Conditional VaR (CVaR)**, i.e. expected loss beyond the VaR threshold

The printed outputs show all key components estimated during this process.

In [26]:
# Download FF3 daily factors and restrict to portfolio date range
ff_factors = fm.load_ff3_factors(start=returns.index[0], end=returns.index[-1])

# Check first few rows
ff_factors.head()

,Mkt_RF,SMB,HML,RF
2024-01-03,-0.0101,-0.0194,-0.0014,0.00022
2024-01-04,-0.0033,0.0021,0.0009,0.00022
2024-01-05,0.0015,-0.0081,0.0065,0.00022
2024-01-08,0.0149,0.0091,-0.0131,0.00022
2024-01-09,-0.0021,-0.0043,-0.0090,0.00022


In [27]:
var_ff, es_ff, ff_betas, ff_resid, ff_cov = fm.ff3_var_cvar(
    prices=prices, shares=shares, alpha=0.99
)

print(f"FF3 VaR  (99%): {var_ff:,.2f} USD")
print(f"FF3 CVaR (99%): {cvar_ff:,.2f} USD\n")

print("FF3 Betas:")
display(ff_betas)

print("\n Idiosyncratic variances:")
display(ff_resid)

print("\n FF3 Covariance Matrix (Σ):")
display(pd.DataFrame(ff_cov, index=ff_betas.index, columns=ff_betas.index))

FF3 VaR  (99%): 72.30 USD
FF3 CVaR (99%): 82.84 USD

FF3 Betas:


,Mkt_RF,SMB,HML
AAPL,0.756763,0.003323,-0.301983
MSFT,1.045876,-0.305718,-0.116980
NVDA,2.310951,-0.930216,-0.890198



 Idiosyncratic variances:


AAPL    0.000315
MSFT    0.000173
NVDA    0.000931
dtype: float64


 FF3 Covariance Matrix (Σ):


,AAPL,MSFT,NVDA
AAPL,0.000360,0.000051,0.000127
MSFT,0.000051,0.000234,0.000150
NVDA,0.000127,0.000150,0.001324
